Recently, we’ve noticed some fluctuations in email marketing campaign performance. Over the last month, our campaigns have been doing very poorly compared to prior months. The leadership team has asked us a few questions about this dip in campaign performance that we’d like you to look into and recommend a course of action. We care about making decisions backed by data and want to ensure that any conclusions we make are meaningful and significant. You have been provided with a data set that contains details of different campaigns we’ve launched and various metrics.
<br>
<br>
Data is in the campaign_performance tab

<br>campaign_id: id of the campaign
<br>date_sent: date the campaign was sent to contacts
<br>n_sent: number of emails sent to contacts
<br>n_open: number of emails opened
<br>n_click: number of email links clicked

Questions:
<br>Should we be concerned with the recent dip in performance? Explain why or why not.
<br>What recommendations do you have about our email marketing strategy based on your findings ?

Cell 1: Setup and Data Loading
This cell contains all necessary imports and initial configurations.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import make_interp_spline
from scipy.ndimage import gaussian_filter1d
from datetime import datetime, timedelta
from scipy import stats
import scipy.stats as stats
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Load the dataset
file_path = "Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv"
df = pd.read_csv(file_path)

In [3]:
daily_stats = df.groupby('date_sent').agg(
    total_sent=('n_sent', 'sum'),
    total_open=('n_open', 'sum'),
    total_click=('n_click', 'sum')
).reset_index()

# Calculate CTR (Click-Through Rate)
daily_stats['ctr_mean'] = daily_stats['total_click'] / daily_stats['total_open']

# Handle any NaN values from division by zero
daily_stats['ctr_mean'] = daily_stats['ctr_mean'].fillna(0)

In [4]:
# Load and prepare data
def load_data():
    df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                     parse_dates=['date_sent'])
    
    # Calculate metrics
    df['open_rate'] = df['n_open'] / df['n_sent']
    df['ctr'] = df['n_click'] / df['n_open']
    
    # Extract month for grouping
    df['month'] = df['date_sent'].dt.month
    df['year'] = df['date_sent'].dt.year
    
    return df

def smooth_data(x, y, smoothing_factor=300):
    """Create smoothed version of data using B-spline interpolation"""
    # Create a finer mesh for smoother curve
    x_smooth = np.linspace(min(x), max(x), smoothing_factor)
    
    # Fit spline
    try:
        spl = make_interp_spline(x, y, k=3)  # type: BSpline
        y_smooth = spl(x_smooth)
        
        # Ensure values stay within reasonable bounds
        if 'rate' in str(y.name).lower():
            y_smooth = np.clip(y_smooth, 0, 1)
        else:
            y_smooth = np.clip(y_smooth, 0, None)
            
        return x_smooth, y_smooth
    except:
        # Fallback to original data if smoothing fails
        return x, y

def create_yoy_comparison():
    # Load data
    df = load_data()
    
    # Calculate monthly aggregates for each year
    monthly_stats = df.groupby(['year', 'month']).agg({
        'n_sent': 'sum',
        'open_rate': 'mean',
        'ctr': 'mean'
    }).reset_index()
    
    # Create subplots with increased spacing
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            '<b>Email Send Volume</b>',
            '<b>Open Rate</b>',
            '<b>Click-Through Rate</b>'
        ),
        vertical_spacing=0.15,
        row_heights=[0.33, 0.33, 0.33]
    )
    
    # Updated colors for a more modern, softer look
    colors = {
        2022: '#7CB9E8',  # Soft blue
        2023: '#F4A460',  # Soft orange
        2024: '#90EE90'   # Soft green
    }
    
    # Add traces for each year
    for year in sorted(monthly_stats['year'].unique()):
        year_data = monthly_stats[monthly_stats['year'] == year]
        
        # Email Send Volume with smooth lines
        x_smooth, y_smooth = smooth_data(year_data['month'], year_data['n_sent'])
        fig.add_trace(
            go.Scatter(
                x=x_smooth,
                y=y_smooth,
                name=str(year),  # Simplified legend name
                line=dict(
                    color=colors[year],
                    width=3,
                    shape='spline',
                    smoothing=1.3
                ),
                hovertemplate=(
                    '<b>Month:</b> %{x:.0f}<br>'
                    '<b>Sends:</b> %{y:,.0f}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year),  # Group by year
                showlegend=True  # Show legend for all years
            ),
            row=1, col=1
        )
        
        # Add original points as markers
        fig.add_trace(
            go.Scatter(
                x=year_data['month'],
                y=year_data['n_sent'],
                name=f'{year} (Actual)',
                mode='markers',
                marker=dict(
                    color=colors[year],
                    size=8,
                    symbol='circle',
                    line=dict(color='white', width=1)
                ),
                showlegend=False,
                hovertemplate=(
                    '<b>Month:</b> %{x}<br>'
                    '<b>Actual Sends:</b> %{y:,.0f}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year)
            ),
            row=1, col=1
        )
        
        # Open Rate with smooth lines
        x_smooth, y_smooth = smooth_data(year_data['month'], year_data['open_rate'])
        fig.add_trace(
            go.Scatter(
                x=x_smooth,
                y=y_smooth,
                name=str(year),
                line=dict(
                    color=colors[year],
                    width=3,
                    shape='spline',
                    smoothing=1.3
                ),
                hovertemplate=(
                    '<b>Month:</b> %{x:.0f}<br>'
                    '<b>Open Rate:</b> %{y:.2%}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year),
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Add original points as markers for Open Rate
        fig.add_trace(
            go.Scatter(
                x=year_data['month'],
                y=year_data['open_rate'],
                name=f'{year} (Actual)',
                mode='markers',
                marker=dict(
                    color=colors[year],
                    size=8,
                    symbol='circle',
                    line=dict(color='white', width=1)
                ),
                showlegend=False,
                hovertemplate=(
                    '<b>Month:</b> %{x}<br>'
                    '<b>Actual Open Rate:</b> %{y:.2%}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year)
            ),
            row=2, col=1
        )
        
        # CTR with smooth lines
        x_smooth, y_smooth = smooth_data(year_data['month'], year_data['ctr'])
        fig.add_trace(
            go.Scatter(
                x=x_smooth,
                y=y_smooth,
                name=str(year),
                line=dict(
                    color=colors[year],
                    width=3,
                    shape='spline',
                    smoothing=1.3
                ),
                hovertemplate=(
                    '<b>Month:</b> %{x:.0f}<br>'
                    '<b>CTR:</b> %{y:.2%}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year),
                showlegend=False
            ),
            row=3, col=1
        )
        
        # Add original points as markers for CTR
        fig.add_trace(
            go.Scatter(
                x=year_data['month'],
                y=year_data['ctr'],
                name=f'{year} (Actual)',
                mode='markers',
                marker=dict(
                    color=colors[year],
                    size=8,
                    symbol='circle',
                    line=dict(color='white', width=1)
                ),
                showlegend=False,
                hovertemplate=(
                    '<b>Month:</b> %{x}<br>'
                    '<b>Actual CTR:</b> %{y:.2%}<br>'
                    f'<b>Year:</b> {year}'
                ),
                legendgroup=str(year)
            ),
            row=3, col=1
        )
    
    # Update layout with improved formatting and spacing
    fig.update_layout(
        title=dict(
            text='<b>Year-Over-Year Email Marketing Performance Comparison</b>',
            x=0.5,
            y=0.97,  # Moved title up slightly
            xanchor='center',
            yanchor='top',
            font=dict(size=24)
        ),
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,  # Increased space between title and legend
            xanchor="center",
            x=0.5,
            bgcolor='rgba(255, 255, 255, 0.9)',
            bordercolor='rgba(240, 240, 240, 1)',  # Lighter border
            borderwidth=1,
            font=dict(size=12),
            itemsizing='constant',  # Ensure consistent legend item sizes
            itemwidth=40,  # Control legend item width
            itemclick=False,  # Disable item clicking
            itemdoubleclick=False  # Disable item double clicking
        ),
        height=1000,
        width=1200,
        template='plotly_white',
        paper_bgcolor='white',
        plot_bgcolor='rgba(250, 250, 250, 0.5)',  # Lighter background
        margin=dict(t=150, b=50, l=80, r=80)  # Increased top margin for legend
    )
    
    # Update axes with softer grid
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for i in range(1, 4):
        # Update x-axes
        fig.update_xaxes(
            title_text="<b>Month</b>",
            ticktext=month_names,
            tickvals=list(range(1, 13)),
            tickangle=0,
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128, 128, 128, 0.1)',  # Softer grid
            showline=True,
            linewidth=1,
            linecolor='rgba(0, 0, 0, 0.1)',  # Softer axis lines
            range=[0.5, 12.5],
            row=i,
            col=1
        )
        
        # Update y-axes
        if i == 1:
            fig.update_yaxes(
                title_text="<b>Number of Emails Sent</b>",
                tickformat=",",
                showgrid=True,
                gridwidth=1,
                gridcolor='rgba(128, 128, 128, 0.1)',  # Softer grid
                showline=True,
                linewidth=1,
                linecolor='rgba(0, 0, 0, 0.1)',  # Softer axis lines
                row=i,
                col=1
            )
        else:
            fig.update_yaxes(
                title_text="<b>Rate</b>",
                tickformat='.1%',
                showgrid=True,
                gridwidth=1,
                gridcolor='rgba(128, 128, 128, 0.1)',  # Softer grid
                showline=True,
                linewidth=1,
                linecolor='rgba(0, 0, 0, 0.1)',  # Softer axis lines
                row=i,
                col=1
            )
    
    # Update annotation style for a more modern look
    for metric in ['n_sent', 'open_rate', 'ctr']:
        for year in range(2023, 2025):
            prev_year_data = monthly_stats[monthly_stats['year'] == year-1]
            curr_year_data = monthly_stats[monthly_stats['year'] == year]
            
            if not prev_year_data.empty and not curr_year_data.empty:
                prev_year_avg = prev_year_data[metric].mean()
                curr_year_avg = curr_year_data[metric].mean()
                yoy_change = (curr_year_avg - prev_year_avg) / prev_year_avg * 100
                
                row = 1 if metric == 'n_sent' else 2 if metric == 'open_rate' else 3
                
                # Add annotation with softer styling
                fig.add_annotation(
                    x=12.2,
                    y=curr_year_data[metric].iloc[-1] if not curr_year_data.empty else 0,
                    text=f'<b>{year} vs {year-1}:</b><br>{yoy_change:+.1f}%',
                    showarrow=False,
                    xanchor='left',
                    yanchor='middle',
                    font=dict(
                        size=11,
                        color=colors[year],
                        family='Arial'
                    ),
                    bgcolor='rgba(255, 255, 255, 0.9)',
                    bordercolor=colors[year],
                    borderwidth=1,
                    borderpad=4,
                    row=row,
                    col=1
                )
    
    return fig

def main():
    # Create the visualization
    fig = create_yoy_comparison()
    
    # Show the plot
    fig.show()

if __name__ == "__main__":
    main() 

In [5]:
def smooth_data(x, y, smoothing_factor=0.8):
    """Create smoothed version of data using B-spline interpolation"""
    # Handle NaN and inf values
    y = pd.Series(y)
    y = y.replace([np.inf, -np.inf], np.nan)
    
    # Interpolate NaN values
    y = y.interpolate(method='linear', limit_direction='both')
    
    # If still have NaN values after interpolation, fill with mean
    if y.isna().any():
        y = y.fillna(y.mean())
    
    # Convert to numpy array
    y = y.to_numpy()
    
    # Ensure minimum length for smoothing
    if len(y) < 4:
        # Return original data if too short
        return x, y
        
    try:
        # Convert dates to numbers for interpolation
        x_numeric = np.arange(len(x))
        
        # Create smooth curve using B-spline
        spl = make_interp_spline(x_numeric, y, k=min(3, len(y)-1))
        
        # Generate smooth points
        x_smooth = np.linspace(0, len(x)-1, 300)
        y_smooth = spl(x_smooth)
        
        # Additional Gaussian smoothing
        y_smooth = gaussian_filter1d(y_smooth, sigma=smoothing_factor*10)
        
        # Map x_smooth back to dates
        date_range = pd.date_range(start=x.min(), end=x.max(), periods=len(x_smooth))
        
        return date_range, y_smooth
    except Exception as e:
        print(f"Smoothing failed: {str(e)}")
        return x, y

def create_enhanced_daily_ctr_analysis(daily_stats):
    """Create an enhanced visualization of daily CTR analysis"""
    try:
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Add smoothed CTR line
        dates_smooth, ctr_mean_smooth = smooth_data(daily_stats['date'], daily_stats['ctr_mean'])
        _, ctr_upper_smooth = smooth_data(daily_stats['date'], daily_stats['ctr_ci_upper'])
        _, ctr_lower_smooth = smooth_data(daily_stats['date'], daily_stats['ctr_ci_lower'])
        _, trend_smooth = smooth_data(daily_stats['date'], daily_stats['ctr_trend'], smoothing_factor=1.2)
        
        # Add smoothed trend for email sends
        _, sends_trend_smooth = smooth_data(daily_stats['date'], daily_stats['total_sent'], smoothing_factor=1.5)

        # Calculate year-over-year growth rates using full year data
        daily_stats['year'] = daily_stats['date'].dt.year
        
        # Group by year and calculate annual metrics
        yearly_stats = daily_stats.groupby('year').agg({
            'total_sent': 'sum',
            'ctr_mean': 'mean'
        }).reset_index()
        
        # Get current and previous year
        current_year = yearly_stats['year'].max()
        previous_year = current_year - 1
        
        # Calculate YoY growth comparing full years
        if previous_year in yearly_stats['year'].values:
            current_year_data = yearly_stats[yearly_stats['year'] == current_year]
            previous_year_data = yearly_stats[yearly_stats['year'] == previous_year]
            
            # Calculate volume growth
            volume_growth = ((current_year_data['total_sent'].iloc[0] - 
                            previous_year_data['total_sent'].iloc[0]) / 
                           previous_year_data['total_sent'].iloc[0] * 100)
            
            # Calculate CTR growth
            ctr_growth = ((current_year_data['ctr_mean'].iloc[0] - 
                          previous_year_data['ctr_mean'].iloc[0]) / 
                         previous_year_data['ctr_mean'].iloc[0] * 100)
            
            # Create period label for annotation
            period_label = f"{previous_year} vs {current_year}"
        else:
            # Fallback if we don't have full previous year data
            volume_growth = 0
            ctr_growth = 0
            period_label = f"{current_year}"

        # Add confidence interval for CTR
        fig.add_trace(
            go.Scatter(
                x=dates_smooth.tolist() + dates_smooth.tolist()[::-1],
                y=ctr_upper_smooth.tolist() + ctr_lower_smooth.tolist()[::-1],
                fill='toself',
                fillcolor='rgba(231,234,241,0.5)',
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo='skip',
                showlegend=True,
                name='95% Confidence Band'
            ),
            secondary_y=True
        )

        # Add email send volume as bars with reduced opacity
        fig.add_trace(
            go.Bar(
                x=daily_stats['date'],
                y=daily_stats['total_sent'],
                name='Daily Emails Sent',
                marker_color='rgba(52, 152, 219, 0.3)',
                hovertemplate='Date: %{x}<br>Sent: %{y:,}<extra></extra>'
            ),
            secondary_y=False
        )

        # Add trend line for email sends
        fig.add_trace(
            go.Scatter(
                x=dates_smooth,
                y=sends_trend_smooth,
                mode='lines',
                line=dict(
                    color='rgba(52, 152, 219, 0.9)',
                    width=3,
                    shape='spline'
                ),
                name='Email Volume Trend'
            ),
            secondary_y=False
        )

        # Add actual CTR values
        fig.add_trace(
            go.Scatter(
                x=daily_stats['date'],
                y=daily_stats['ctr_mean'],
                mode='markers',
                marker=dict(
                    size=6,
                    color='rgba(255, 65, 54, 0.5)',
                ),
                name='Daily CTR',
                hovertemplate='Date: %{x}<br>CTR: %{y:.2%}<extra></extra>'
            ),
            secondary_y=True
        )

        # Add smoothed trend line for CTR
        fig.add_trace(
            go.Scatter(
                x=dates_smooth,
                y=trend_smooth,
                mode='lines',
                line=dict(
                    color='rgba(255, 65, 54, 0.8)',
                    width=3,
                    shape='spline'
                ),
                name='CTR Trend'
            ),
            secondary_y=True
        )

        # Update layout with modern styling
        fig.update_layout(
            title=dict(
                text='Daily Click-to-Open Rate and Send Volume Analysis<br><sup>Showing parallel growth in both metrics</sup>',
                x=0.5,
                y=0.95,
                xanchor='center',
                yanchor='top',
                font=dict(size=20)
            ),
            xaxis_title='Date',
            yaxis_title='Number of Emails Sent',
            yaxis2_title='Click-to-Open Rate (CTR)',
            template='plotly_white',
            height=700,
            width=1200,
            hovermode='x unified',
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01,
                bgcolor='rgba(255, 255, 255, 0.8)',
                bordercolor='rgba(0, 0, 0, 0.1)',
                borderwidth=1
            ),
            margin=dict(l=60, r=60, t=100, b=60),  # Reduced bottom margin
            plot_bgcolor='rgba(250,250,250,0.9)',
            paper_bgcolor='white'
        )

        # Add a semi-transparent background box for annotations
        fig.add_shape(
            type="rect",
            x0=0.35,
            y0=0.85,
            x1=0.65,
            y1=0.95,
            xref="paper",
            yref="paper",
            fillcolor="white",
            opacity=0.8,
            layer="below",
            line_width=0,
        )

        # Add title for growth metrics in the center
        fig.add_annotation(
            x=0.5,
            y=0.92,
            xref='paper',
            yref='paper',
            text=f"<b>Year-over-Year Growth</b>",
            showarrow=False,
            font=dict(
                size=14,
                color='black'
            ),
            align='center'
        )

        # Add volume growth annotation
        fig.add_annotation(
            x=0.4,
            y=0.88,
            xref='paper',
            yref='paper',
            text=f"<span style='color: rgba(52, 152, 219, 1)'>Volume</span> ({period_label}):<br><b>{volume_growth:+.1f}%</b>",
            showarrow=False,
            font=dict(
                size=12,
                color='black'
            ),
            align='center'
        )

        # Add CTR growth annotation
        fig.add_annotation(
            x=0.6,
            y=0.88,
            xref='paper',
            yref='paper',
            text=f"<span style='color: rgba(255, 65, 54, 1)'>CTR</span> ({period_label}):<br><b>{ctr_growth:+.1f}%</b>",
            showarrow=False,
            font=dict(
                size=12,
                color='black'
            ),
            align='center'
        )

        # Update axes with modern styling
        fig.update_xaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.1)',
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='rgba(0,0,0,0.2)'
        )
        
        # Update primary y-axis (email volume)
        fig.update_yaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.1)',
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='rgba(0,0,0,0.2)',
            secondary_y=False
        )
        
        # Update secondary y-axis (CTR)
        fig.update_yaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.1)',
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='rgba(0,0,0,0.2)',
            tickformat='.1%',
            secondary_y=True
        )

        return fig

    except Exception as e:
        print(f"Error creating visualization: {str(e)}")
        return None

def main():
    # Load and prepare data
    df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                     parse_dates=['date_sent'])
    
    # Calculate metrics
    df['open_rate'] = df['n_open']/df['n_sent']
    df['ctr'] = np.where(df['n_open'] > 0, df['n_click']/df['n_open'], 0)
    
    # Create daily stats
    daily_stats = df.groupby('date_sent').agg({
        'n_sent': 'sum',
        'n_open': 'sum',
        'n_click': 'sum'
    }).reset_index()
    
    daily_stats.columns = ['date', 'total_sent', 'total_opened', 'total_clicks']
    daily_stats['ctr_mean'] = daily_stats['total_clicks'] / daily_stats['total_sent']
    daily_stats['ctr_se'] = daily_stats['ctr_mean'] / np.sqrt(daily_stats['total_sent'])
    daily_stats['ctr_ci_lower'] = daily_stats['ctr_mean'] - 1.96 * daily_stats['ctr_se']
    daily_stats['ctr_ci_upper'] = daily_stats['ctr_mean'] + 1.96 * daily_stats['ctr_se']
    
    # Calculate trend
    window_length = min(15, len(daily_stats) - (len(daily_stats) % 2 - 1))
    if window_length > 2:
        daily_stats['ctr_trend'] = gaussian_filter1d(daily_stats['ctr_mean'], sigma=3)
    else:
        daily_stats['ctr_trend'] = daily_stats['ctr_mean'].rolling(window=3, center=True).mean()
    
    # Create and show visualization
    fig = create_enhanced_daily_ctr_analysis(daily_stats)
    if fig:
        fig.show()

if __name__ == "__main__":
    main() 

In [6]:
# Load and prepare data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                 parse_dates=['date_sent'])

# Calculate metrics
df['open_rate'] = df['n_open']/df['n_sent']
df['ctr'] = np.where(df['n_open'] > 0, df['n_click']/df['n_open'], 0)

# Define recent period (last 30 days) and historical (all data)
recent_cutoff = df['date_sent'].max() - timedelta(days=30)  # Changed to 30 days
recent_data = df[df['date_sent'] >= recent_cutoff]
historical_data = df  # All data for historical comparison

def perform_statistical_comparison(metric):
    """Perform comprehensive statistical analysis for a given metric"""
    # Perform t-test
    t_stat, t_pval = stats.ttest_ind(
        recent_data[metric],
        historical_data[metric]
    )
    
    # Perform Mann-Whitney U test (non-parametric)
    u_stat, u_pval = mannwhitneyu(
        recent_data[metric],
        historical_data[metric],
        alternative='two-sided'
    )
    
    # Calculate Cohen's d effect size
    pooled_std = np.sqrt((recent_data[metric].var() + historical_data[metric].var()) / 2)
    cohens_d = (recent_data[metric].mean() - historical_data[metric].mean()) / pooled_std
    
    # Calculate confidence intervals
    recent_ci = stats.t.interval(
        0.95,
        len(recent_data[metric])-1,
        loc=recent_data[metric].mean(),
        scale=stats.sem(recent_data[metric])
    )
    
    historical_ci = stats.t.interval(
        0.95,
        len(historical_data[metric])-1,
        loc=historical_data[metric].mean(),
        scale=stats.sem(historical_data[metric])
    )
    
    return {
        'metric': metric,
        'recent_mean': recent_data[metric].mean(),
        'historical_mean': historical_data[metric].mean(),
        'recent_std': recent_data[metric].std(),
        'historical_std': historical_data[metric].std(),
        'change': (recent_data[metric].mean() - historical_data[metric].mean()) / historical_data[metric].mean(),
        't_stat': t_stat,
        't_pvalue': t_pval,
        'u_stat': u_stat,
        'u_pvalue': u_pval,
        'cohens_d': cohens_d,
        'recent_ci': recent_ci,
        'historical_ci': historical_ci,
        'significant': (t_pval < 0.05) or (u_pval < 0.05)  # Significant if either test shows significance
    }

# Perform analysis for both metrics
metrics = ['open_rate', 'ctr']
results = {metric: perform_statistical_comparison(metric) for metric in metrics}

# Create the visualization
fig = go.Figure()

# Set up the bar positions
bar_positions = np.array([1, 2, 4, 5])
bar_width = 0.8

# Colors
historical_color = 'rgba(52, 152, 219, 0.7)'  # Blue
recent_color = 'rgba(46, 204, 113, 0.7)'      # Green

# Add bars for both metrics
for i, metric in enumerate(['open_rate', 'ctr']):
    # Historical bar
    fig.add_trace(go.Bar(
        x=[bar_positions[i*2]],
        y=[results[metric]['historical_mean']],
        name='Historical',
        marker_color=historical_color,
        width=bar_width,
        text=[f"{results[metric]['historical_mean']:.1%}"],
        textposition='auto',
        showlegend=False
    ))

    # Recent bar
    fig.add_trace(go.Bar(
        x=[bar_positions[i*2 + 1]],
        y=[results[metric]['recent_mean']],
        name='Recent (Last 30 Days)',  # Updated text
        marker_color=recent_color,
        width=bar_width,
        text=[f"{results[metric]['recent_mean']:.1%}"],
        textposition='auto',
        showlegend=False
    ))

# Add statistical annotations
for i, metric in enumerate(['open_rate', 'ctr']):
    x_pos = bar_positions[i*2] + 0.5
    y_pos = max(results[metric]['historical_mean'], 
                results[metric]['recent_mean']) * 1.05  # Adjusted position
    
    # Use minimum p-value from both tests
    p_val = min(results[metric]['t_pvalue'], results[metric]['u_pvalue'])
    
    # Add p-value and significance annotation
    fig.add_annotation(
        x=x_pos,
        y=y_pos,
        text=f"p={p_val:.2992}" + ("<br>*" if results[metric]['significant'] else ""),  # Added line break and made star more visible
        showarrow=False,
        font=dict(size=14, color='rgba(0,0,0,0.8)'),
        align='center',
        yshift=20  # Increased vertical spacing
    )

# Update layout
fig.update_layout(
    title=dict(
        text='Statistical Comparison: Recent (30 Days) vs Historical Performance',  # Updated title
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20)
    ),
    xaxis=dict(
        ticktext=['Open Rate Historical', 'Open Rate Recent', 
                 'CTR Historical', 'CTR Recent'],
        tickvals=bar_positions,
        tickangle=0
    ),
    yaxis=dict(
        title='Rate',
        tickformat='.1%',
        gridcolor='rgba(128,128,128,0.1)',
        zerolinecolor='rgba(128,128,128,0.1)',
        range=[0, max(max(results['open_rate']['historical_mean'], 
                         results['open_rate']['recent_mean']) * 1.4,  # Increased range to accommodate annotations
                     max(results['ctr']['historical_mean'],
                         results['ctr']['recent_mean']) * 1.4)]
    ),
    plot_bgcolor='white',
    showlegend=False,
    width=1000,
    height=600,
    margin=dict(t=100, b=50, l=50, r=50),
    annotations=[
        dict(
            text="* indicates statistical significance (p < 0.05)",
            xref="paper",
            yref="paper",
            x=0,
            y=-0.15,
            showarrow=False,
            font=dict(size=10, color='gray')
        )
    ]
)

# Add percentage change annotations with significance indicator
for i, metric in enumerate(['open_rate', 'ctr']):
    change = results[metric]['change'] * 100
    x_pos = bar_positions[i*2] + 0.5
    y_pos = max(results[metric]['historical_mean'], 
                results[metric]['recent_mean']) * 1.25  # Adjusted position
    
    # Add significance indicator to the change text
    change_text = f"{change:+.1f}% change" + (" *" if results[metric]['significant'] else "")
    
    fig.add_annotation(
        x=x_pos,
        y=y_pos,
        text=change_text,
        showarrow=False,
        font=dict(size=12, color='gray'),
        yshift=10
    )

# Show the figure
fig.show()

# Print detailed analysis
print("\nComprehensive Statistical Analysis:")
print("=" * 80)
for metric in metrics:
    result = results[metric]
    print(f"\n{metric.upper()}:")
    print("-" * 50)
    print(f"Historical Mean (95% CI): {result['historical_mean']:.1%} ({result['historical_ci'][0]:.1%} to {result['historical_ci'][1]:.1%})")
    print(f"Recent Mean (95% CI): {result['recent_mean']:.1%} ({result['recent_ci'][0]:.1%} to {result['recent_ci'][1]:.1%})")
    print(f"Absolute Change: {result['change']*100:+.1f}%")
    print("\nStatistical Tests:")
    print(f"T-test: t={result['t_stat']:.2f}, p={result['t_pvalue']:.4f}")
    print(f"Mann-Whitney U test: U={result['u_stat']:.2f}, p={result['u_pvalue']:.4f}")
    print(f"Effect Size (Cohen's d): {result['cohens_d']:.2f}")
    print(f"Effect Size Interpretation: {
        'Very Large' if abs(result['cohens_d']) > 1.2 else
        'Large' if abs(result['cohens_d']) > 0.8 else
        'Medium' if abs(result['cohens_d']) > 0.5 else
        'Small' if abs(result['cohens_d']) > 0.2 else
        'Negligible'
    }")
    print(f"\nStatistically Significant: {'Yes' if result['significant'] else 'No'}")
    if result['significant']:
        print("Note: Significant changes detected in both parametric and/or non-parametric tests") 


Comprehensive Statistical Analysis:

OPEN_RATE:
--------------------------------------------------
Historical Mean (95% CI): 30.0% (29.7% to 30.3%)
Recent Mean (95% CI): 26.9% (24.2% to 29.7%)
Absolute Change: -10.1%

Statistical Tests:
T-test: t=-2.67, p=0.0079
Mann-Whitney U test: U=266.50, p=0.0135
Effect Size (Cohen's d): -1.28
Effect Size Interpretation: Very Large

Statistically Significant: Yes
Note: Significant changes detected in both parametric and/or non-parametric tests

CTR:
--------------------------------------------------
Historical Mean (95% CI): 13.7% (13.5% to 13.9%)
Recent Mean (95% CI): 14.4% (13.7% to 15.0%)
Absolute Change: +5.0%

Statistical Tests:
T-test: t=0.89, p=0.3732
Mann-Whitney U test: U=920.50, p=0.3847
Effect Size (Cohen's d): 0.54
Effect Size Interpretation: Medium

Statistically Significant: No


In [7]:
# Load and prepare data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                 parse_dates=['date_sent'])

# Calculate metrics
df['open_rate'] = df['n_open']/df['n_sent']
df['ctr'] = np.where(df['n_open'] > 0, df['n_click']/df['n_open'], 0)
df['year'] = df['date_sent'].dt.year
df['day_of_year'] = df['date_sent'].dt.dayofyear

# Get the most recent date and calculate the day range for comparison
max_date = df['date_sent'].max()
current_day_of_year = max_date.dayofyear
start_day = current_day_of_year - 30

# Get data for each year's same period
def get_period_data(year, start_day, end_day):
    year_data = df[df['year'] == year]
    return year_data[
        (year_data['day_of_year'] >= start_day) & 
        (year_data['day_of_year'] <= end_day)
    ]

recent_data = get_period_data(2024, start_day, current_day_of_year)
data_2023 = get_period_data(2023, start_day, current_day_of_year)
data_2022 = get_period_data(2022, start_day, current_day_of_year)
historical_data = df  # All data for historical comparison

def perform_statistical_comparison(metric, data1, data2, period1_name, period2_name):
    """Perform comprehensive statistical analysis for a given metric between two periods"""
    if len(data1) == 0 or len(data2) == 0:
        return None
        
    # Perform t-test
    t_stat, t_pval = stats.ttest_ind(
        data1[metric],
        data2[metric]
    )
    
    # Perform Mann-Whitney U test (non-parametric)
    u_stat, u_pval = mannwhitneyu(
        data1[metric],
        data2[metric],
        alternative='two-sided'
    )
    
    # Calculate Cohen's d effect size
    pooled_std = np.sqrt((data1[metric].var() + data2[metric].var()) / 2)
    cohens_d = (data1[metric].mean() - data2[metric].mean()) / pooled_std
    
    return {
        'metric': metric,
        f'{period1_name}_mean': data1[metric].mean(),
        f'{period2_name}_mean': data2[metric].mean(),
        'change': (data1[metric].mean() - data2[metric].mean()) / data2[metric].mean(),
        't_stat': t_stat,
        't_pvalue': t_pval,
        'u_stat': u_stat,
        'u_pvalue': u_pval,
        'cohens_d': cohens_d,
        'significant': (t_pval < 0.05) or (u_pval < 0.05)
    }

# Perform analysis for both metrics
metrics = ['open_rate', 'ctr']
results = {}
for metric in metrics:
    # Current vs Historical
    results[f'{metric}_current_vs_hist'] = perform_statistical_comparison(
        metric, recent_data, historical_data, 'recent', 'historical'
    )
    # Current vs 2023
    results[f'{metric}_current_vs_2023'] = perform_statistical_comparison(
        metric, recent_data, data_2023, 'recent', '2023'
    )
    # Current vs 2022
    results[f'{metric}_current_vs_2022'] = perform_statistical_comparison(
        metric, recent_data, data_2022, 'recent', '2022'
    )

# Create the visualization
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Open Rate Comparison', 'CTR Comparison'),
                    horizontal_spacing=0.1)

# Colors for different years
colors = {
    'Historical': 'rgba(52, 152, 219, 0.7)',  # Blue
    '2022': 'rgba(155, 89, 182, 0.7)',        # Purple
    '2023': 'rgba(46, 204, 113, 0.7)',        # Green
    '2024': 'rgba(230, 126, 34, 0.7)'         # Orange
}

# Add bars for each metric
for i, metric in enumerate(['open_rate', 'ctr']):
    col = i + 1
    x_positions = [1, 2, 3, 4]  # One position for each period
    
    # Historical bar
    hist_mean = results[f'{metric}_current_vs_hist']['historical_mean']
    fig.add_trace(
        go.Bar(
            x=[x_positions[0]],
            y=[hist_mean],
            name='Historical',
            marker_color=colors['Historical'],
            text=[f"{hist_mean:.1%}"],
            textposition='auto',
            showlegend=False
        ),
        row=1, col=col
    )
    
    # 2022 bar
    if len(data_2022) > 0:
        mean_2022 = data_2022[metric].mean()
        fig.add_trace(
            go.Bar(
                x=[x_positions[1]],
                y=[mean_2022],
                name='2022',
                marker_color=colors['2022'],
                text=[f"{mean_2022:.1%}"],
                textposition='auto',
                showlegend=False
            ),
            row=1, col=col
        )
    
    # 2023 bar
    if len(data_2023) > 0:
        mean_2023 = data_2023[metric].mean()
        fig.add_trace(
            go.Bar(
                x=[x_positions[2]],
                y=[mean_2023],
                name='2023',
                marker_color=colors['2023'],
                text=[f"{mean_2023:.1%}"],
                textposition='auto',
                showlegend=False
            ),
            row=1, col=col
        )
    
    # 2024 (Recent) bar
    recent_mean = results[f'{metric}_current_vs_hist']['recent_mean']
    fig.add_trace(
        go.Bar(
            x=[x_positions[3]],
            y=[recent_mean],
            name='2024',
            marker_color=colors['2024'],
            text=[f"{recent_mean:.1%}"],
            textposition='auto',
            showlegend=False
        ),
        row=1, col=col
    )
    
    # Add significance annotations
    comparisons = [
        ('current_vs_hist', 0.5, 'Historical'),
        ('current_vs_2022', 1.5, '2022'),
        ('current_vs_2023', 2.5, '2023')
    ]
    
    # Calculate maximum y value for the current metric
    max_y = max(
        hist_mean,
        mean_2022 if len(data_2022) > 0 else 0,
        mean_2023 if len(data_2023) > 0 else 0,
        recent_mean
    )

    # Adjust y-axis range based on the metric
    y_range_multiplier = 1.45 if metric == 'open_rate' else 1.5
    
    # Add annotations with better spacing
    for idx, (comp_key, x_pos, label) in enumerate(comparisons):
        result = results[f'{metric}_{comp_key}']
        if result is not None:
            p_val = min(result['t_pvalue'], result['u_pvalue'])
            change = result['change'] * 100
            
            # Calculate y position with better spacing
            # Adjust spacing for open_rate to be more compact
            spacing_factor = 0.08 if metric == 'open_rate' else 0.1
            y_pos = max_y * (1.15 + idx * spacing_factor)
            
            # Format the annotation text more clearly
            sig_star = "*" if result['significant'] else ""
            annotation_text = (
                f"vs {label}:<br>"
                f"Δ{change:+.1f}%<br>"
                f"p={p_val:.3f}{sig_star}"
            )
            
            # Add connecting line to make it clear which comparison it refers to
            fig.add_shape(
                type="line",
                x0=4,  # Position of 2024 bar
                y0=recent_mean,
                x1=x_pos + 0.5,  # End at annotation
                y1=y_pos,
                line=dict(
                    color="rgba(128, 128, 128, 0.3)",
                    width=1,
                    dash="dot"
                ),
                row=1, col=col
            )
            
            # Add annotation with adjusted position and style
            fig.add_annotation(
                x=x_pos + 0.5,
                y=y_pos,
                text=annotation_text,
                showarrow=False,
                font=dict(size=11),
                xref=f'x{col}',
                yref=f'y{col}',
                align='left',
                bgcolor='rgba(255, 255, 255, 0.8)',
                bordercolor='rgba(128, 128, 128, 0.3)',
                borderwidth=1,
                borderpad=4
            )

# Update layout
fig.update_layout(
    title=dict(
        text='Year-over-Year Comparison (Same 30-day Period)',
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20)
    ),
    height=700,  # Increased height to accommodate annotations
    width=1200,
    plot_bgcolor='white',
    margin=dict(t=100, r=200, b=100, l=50)  # Increased right margin for annotations
)

# Update axes with more space for annotations and different ranges for each metric
for i in range(1, 3):
    metric = 'open_rate' if i == 1 else 'ctr'
    max_y = max(
        results[f'{metric}_current_vs_hist']['historical_mean'],
        data_2022[metric].mean() if len(data_2022) > 0 else 0,
        data_2023[metric].mean() if len(data_2023) > 0 else 0,
        results[f'{metric}_current_vs_hist']['recent_mean']
    )
    
    # Set different ranges for open_rate and ctr
    y_range_multiplier = 1.45 if metric == 'open_rate' else 1.5
    
    fig.update_xaxes(
        ticktext=['Historical', '2022', '2023', '2024'],
        tickvals=[1, 2, 3, 4],
        row=1, col=i
    )
    fig.update_yaxes(
        title='Rate',
        tickformat='.1%',
        gridcolor='rgba(128,128,128,0.1)',
        zerolinecolor='rgba(128,128,128,0.1)',
        range=[0, max_y * y_range_multiplier],  # Different range multiplier for each metric
        row=1, col=i
    )

# Add footnote
fig.add_annotation(
    text="* indicates statistical significance (p < 0.05)",
    xref="paper",
    yref="paper",
    x=0,
    y=-0.15,
    showarrow=False,
    font=dict(size=10, color='gray')
)

# Show the figure
fig.show()

# Print detailed analysis
print("\nYear-over-Year Analysis (Same 30-day Period)")
print("=" * 80)

for metric in metrics:
    metric_name = "OPEN RATE" if metric == "open_rate" else "CLICK-THROUGH RATE (CTR)"
    print(f"\n{metric_name}:")
    print("-" * 50)
    
    # Print means for each period
    print(f"Mean Values:")
    print(f"  Historical: {results[f'{metric}_current_vs_hist']['historical_mean']:.1%}")
    if len(data_2022) > 0:
        print(f"  2022: {data_2022[metric].mean():.1%}")
    if len(data_2023) > 0:
        print(f"  2023: {data_2023[metric].mean():.1%}")
    print(f"  2024: {results[f'{metric}_current_vs_hist']['recent_mean']:.1%}")
    
    print("\nStatistical Comparisons (2024 vs):")
    comparisons = [
        ('current_vs_hist', 'Historical'),
        ('current_vs_2022', '2022'),
        ('current_vs_2023', '2023')
    ]
    
    for comp_key, label in comparisons:
        result = results[f'{metric}_{comp_key}']
        if result is not None:
            sig_mark = "*" if result['significant'] else ""
            print(f"\n  {label}:")
            print(f"    Change: {result['change']*100:+.1f}%")
            print(f"    P-value: {min(result['t_pvalue'], result['u_pvalue']):.4f}{sig_mark}")
            print(f"    Effect Size: {abs(result['cohens_d']):.2f} ({'very large' if abs(result['cohens_d']) > 1.2 else 'large' if abs(result['cohens_d']) > 0.8 else 'medium' if abs(result['cohens_d']) > 0.5 else 'small'})") 


Year-over-Year Analysis (Same 30-day Period)

OPEN RATE:
--------------------------------------------------
Mean Values:
  Historical: 30.0%
  2022: 24.4%
  2023: 25.5%
  2024: 26.9%

Statistical Comparisons (2024 vs):

  Historical:
    Change: -10.1%
    P-value: 0.0079*
    Effect Size: 1.28 (very large)

  2022:
    Change: +10.4%
    P-value: 0.0616
    Effect Size: 1.18 (large)

  2023:
    Change: +5.6%
    P-value: 0.1489
    Effect Size: 0.81 (large)

CLICK-THROUGH RATE (CTR):
--------------------------------------------------
Mean Values:
  Historical: 13.7%
  2022: 11.3%
  2023: 12.6%
  2024: 14.4%

Statistical Comparisons (2024 vs):

  Historical:
    Change: +5.0%
    P-value: 0.3732
    Effect Size: 0.54 (medium)

  2022:
    Change: +27.0%
    P-value: 0.0000*
    Effect Size: 4.94 (very large)

  2023:
    Change: +14.3%
    P-value: 0.0001*
    Effect Size: 3.38 (very large)


In [8]:
# Function to format large numbers
def format_number(num):
    if num >= 1_000_000:
        return f'{num/1_000_000:.1f}M'
    elif num >= 1_000:
        return f'{num/1_000:.1f}K'
    return f'{num:.0f}'

def format_change(value):
    return f"{'↑' if value > 0 else '↓'} {abs(value):.1f}%"

# Read the data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                 parse_dates=['date_sent'])
df['year'] = df['date_sent'].dt.year

def create_funnel_data(year_data):
    funnel_stages = [
        {'stage': 'Sent', 'count': year_data['n_sent'].sum()},
        {'stage': 'Opened', 'count': year_data['n_open'].sum()},
        {'stage': 'Clicked', 'count': year_data['n_click'].sum()}
    ]
    funnel_data = pd.DataFrame(funnel_stages)
    
    # Calculate rates
    funnel_data['total_rate'] = funnel_data['count'] / funnel_data['count'].iloc[0]
    funnel_data['step_rate'] = funnel_data['count'].pct_change(-1).fillna(0)
    
    return funnel_data

# Get unique years
years = sorted(df['year'].unique())

# Create the main figure with subplots
fig = make_subplots(
    rows=1, cols=len(years),
    subplot_titles=[f'<b>{year}</b>' for year in years],
    specs=[[{'type': 'funnel'} for _ in years]]
)

# Colors for consistent branding
colors = ['#3498DB', '#2ECC71', '#F1C40F']

# Store metrics for comparison
yearly_metrics = []

# Create funnel for each year
for idx, year in enumerate(years, 1):
    year_data = df[df['year'] == year]
    funnel_data = create_funnel_data(year_data)
    
    # Store metrics for later use
    metrics = {
        'year': year,
        'sent': funnel_data['count'].iloc[0],
        'opened': funnel_data['count'].iloc[1],
        'clicked': funnel_data['count'].iloc[2],
        'open_rate': funnel_data['total_rate'].iloc[1] * 100,
        'click_rate': (funnel_data['count'].iloc[2] / funnel_data['count'].iloc[1]) * 100,
        'overall_rate': funnel_data['total_rate'].iloc[2] * 100
    }
    yearly_metrics.append(metrics)
    
    # Add funnel trace
    fig.add_trace(
        go.Funnel(
            name=str(year),
            y=funnel_data['stage'],
            x=funnel_data['count'],
            textposition="inside",
            textinfo="value+percent initial",
            texttemplate=[
                f"<b>{format_number(funnel_data['count'].iloc[0])}</b><br>({funnel_data['total_rate'].iloc[0]:.1%})",
                f"<b>{format_number(funnel_data['count'].iloc[1])}</b><br>({funnel_data['total_rate'].iloc[1]:.1%})",
                f"<b>{format_number(funnel_data['count'].iloc[2])}</b><br>({funnel_data['total_rate'].iloc[2]:.1%})"
            ],
            textfont=dict(family="Arial", size=14, color="white"),
            opacity=0.9,
            marker={
                "color": colors,
                "line": {"width": [1, 1, 1], "color": ["white", "white", "white"]}
            }
        ),
        row=1, col=idx
    )

# Calculate year-over-year changes
yoy_changes = []
for i in range(1, len(yearly_metrics)):
    current = yearly_metrics[i]
    previous = yearly_metrics[i-1]
    
    volume_change = (current['sent'] - previous['sent']) / previous['sent'] * 100
    open_rate_change = current['open_rate'] - previous['open_rate']
    click_rate_change = current['click_rate'] - previous['click_rate']
    
    yoy_changes.append({
        'years': f"{current['year']}/{previous['year']}",
        'volume': format_change(volume_change),
        'open_rate': format_change(open_rate_change),
        'click_rate': format_change(click_rate_change)
    })

# Add metrics boxes for each year
for idx, metrics in enumerate(yearly_metrics):
    fig.add_annotation(
        x=0.2 + (idx * 0.3),
        y=-0.15,
        text=f"<b>Metrics {metrics['year']}</b><br>" + \
             f"Open Rate: {metrics['open_rate']:.1f}%<br>" + \
             f"Click Rate: {metrics['click_rate']:.1f}%",
        showarrow=False,
        font=dict(size=12, color="#34495E"),
        xref='paper',
        yref='paper',
        align='center',
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1,
        borderpad=4
    )

# Add YoY comparison boxes between years
for idx, change in enumerate(yoy_changes):
    fig.add_annotation(
        x=0.35 + (idx * 0.3),
        y=-0.3,
        text=f"<b>{change['years']} Changes</b><br>" + \
             f"Volume: {change['volume']}<br>" + \
             f"Open Rate: {change['open_rate']}<br>" + \
             f"Click Rate: {change['click_rate']}",
        showarrow=False,
        font=dict(size=12, color="#34495E"),
        xref='paper',
        yref='paper',
        align='center',
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1,
        borderpad=4
    )

# Update layout with modern styling
fig.update_layout(
    title=dict(
        text='Email Marketing Performance Evolution<br>' +
             '<span style="font-size: 14px; color: #666;">Volume and Engagement Metrics Year-over-Year</span>',
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=24, family="Arial", color="#2C3E50")
    ),
    showlegend=False,
    height=650,  # Reduced height
    width=300 * len(years),
    template='plotly_white',
    margin=dict(l=30, r=30, t=120, b=120),  # Adjusted bottom margin
    paper_bgcolor='white',
    plot_bgcolor='rgba(240,240,240,0.0)',
    font=dict(family="Arial", size=12, color="#2C3E50")
)
# Show the figure
fig.show() 

# Print concise summary with data validation
print("\nEmail Marketing Performance Summary")
print("=" * 40)

for metrics in yearly_metrics:
    print(f"\n{metrics['year']} Performance:")
    print(f"Volume: {format_number(metrics['sent'])} emails sent")
    print(f"Opened: {format_number(metrics['opened'])} ({metrics['open_rate']:.1f}%)")
    print(f"Clicked: {format_number(metrics['clicked'])} ({metrics['click_rate']:.1f}%)")
    print(f"Overall Conversion: {metrics['overall_rate']:.1f}%")

print("\nYear-over-Year Changes")
print("=" * 40)
for change in yoy_changes:
    print(f"\n{change['years']}:")
    print(f"Volume: {change['volume']}")
    print(f"Open Rate: {change['open_rate']}")
    print(f"Click Rate: {change['click_rate']}")




Email Marketing Performance Summary

2022 Performance:
Volume: 1.1M emails sent
Opened: 296.2K (28.1%)
Clicked: 34.4K (11.6%)
Overall Conversion: 3.3%

2023 Performance:
Volume: 1.3M emails sent
Opened: 401.2K (29.8%)
Clicked: 54.9K (13.7%)
Overall Conversion: 4.1%

2024 Performance:
Volume: 1.4M emails sent
Opened: 432.7K (31.9%)
Clicked: 66.8K (15.4%)
Overall Conversion: 4.9%

Year-over-Year Changes

2023/2022:
Volume: ↑ 27.6%
Open Rate: ↑ 1.7%
Click Rate: ↑ 2.1%

2024/2023:
Volume: ↑ 0.7%
Open Rate: ↑ 2.1%
Click Rate: ↑ 1.8%


In [9]:
# Read the data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', 
                 parse_dates=['date_sent'])

# Calculate rates as percentages
df['ctr'] = (df['n_click'] / df['n_open']) * 100
df['open_rate'] = (df['n_open'] / df['n_sent']) * 100

# Calculate performance thresholds for both metrics
metrics = {
    'ctr': {
        'name': 'CTR',
        'data': df['ctr'],
        'percentiles': {
            '25th': df['ctr'].quantile(0.25),
            'Median': df['ctr'].quantile(0.50),
            '75th': df['ctr'].quantile(0.75)
        },
        'range': [
            max(0, df['ctr'].min() - 0.1 * (df['ctr'].max() - df['ctr'].min())),
            df['ctr'].max() + 0.1 * (df['ctr'].max() - df['ctr'].min())
        ]
    },
    'open_rate': {
        'name': 'Open Rate',
        'data': df['open_rate'],
        'percentiles': {
            '25th': df['open_rate'].quantile(0.25),
            'Median': df['open_rate'].quantile(0.50),
            '75th': df['open_rate'].quantile(0.75)
        },
        'range': [
            max(0, df['open_rate'].min() - 0.1 * (df['open_rate'].max() - df['open_rate'].min())),
            df['open_rate'].max() + 0.1 * (df['open_rate'].max() - df['open_rate'].min())
        ]
    }
}

# Create subplots with more compact spacing
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'CTR Distribution with Performance Thresholds',
        'Open Rate Distribution with Performance Thresholds'
    ),
    vertical_spacing=0.16
)

# Colors for percentile lines and styling
colors = {
    '25th': '#E74C3C',  # Red
    'Median': '#F39C12',  # Orange
    '75th': '#27AE60'  # Green
}

# Create plots for each metric
for idx, (metric_key, metric) in enumerate(metrics.items(), 1):
    # Create histogram with optimal number of bins
    hist, bins = np.histogram(metric['data'], bins='auto')
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Add histogram with improved styling
    fig.add_trace(
        go.Bar(
            x=bin_centers,
            y=hist,
            name='Distribution',
            marker_color='rgba(144, 238, 144, 0.7)',
            marker_line_color='rgba(0, 100, 0, 0.3)',
            marker_line_width=1,
            hovertemplate=f'{metric["name"]}: %{{x:.1f}}%<br>Count: %{{y}}<extra></extra>',
            showlegend=(idx == 1)
        ),
        row=idx, col=1
    )
    
    # Add KDE with improved smoothing
    kde_x = np.linspace(metric['range'][0], metric['range'][1], 200)
    kde = stats.gaussian_kde(metric['data'], bw_method='silverman')
    kde_y = kde(kde_x) * len(metric['data']) * (bins[1] - bins[0])
    
    fig.add_trace(
        go.Scatter(
            x=kde_x,
            y=kde_y,
            name='Density Curve',
            line=dict(color='#2ECC71', width=2.5, shape='spline'),
            hovertemplate=f'{metric["name"]}: %{{x:.1f}}%<br>Density: %{{y:.1f}}<extra></extra>',
            showlegend=(idx == 1)
        ),
        row=idx, col=1
    )
    
    # Calculate max y value for positioning annotations
    max_y = max(max(hist), max(kde_y))
    
    # Add percentile labels at the top of the plot
    fig.add_annotation(
        x=metric['range'][0],
        y=max_y * 1.15,
        text=f"<b>Performance Thresholds:</b> " +
             f"<span style='color: {colors['25th']}'>25th: {metric['percentiles']['25th']:.1f}%</span>  " +
             f"<span style='color: {colors['Median']}'>Median: {metric['percentiles']['Median']:.1f}%</span>  " +
             f"<span style='color: {colors['75th']}'>75th: {metric['percentiles']['75th']:.1f}%</span>",
        showarrow=False,
        xref=f'x{idx}',
        yref=f'y{idx}',
        xanchor='left',
        font=dict(size=11),
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1,
        borderpad=4
    )
    
    # Add vertical lines for percentiles without text annotations
    for label, value in metric['percentiles'].items():
        fig.add_vline(
            x=value,
            line_dash="dash",
            line_color=colors[label],
            line_width=1.5,
            row=idx, col=1
        )
    
    # Add stats annotation
    stats_text = (
        f"<b>Key Statistics:</b><br>" +
        f"Mean: {metric['data'].mean():.1f}%<br>" +
        f"Std Dev: {metric['data'].std():.1f}%<br>" +
        f"Sample Size: {len(df):,}"
    )
    
    fig.add_annotation(
        x=0.99,
        y=0.95,
        xref=f'x{idx}',
        yref=f'y{idx}',
        text=stats_text,
        showarrow=False,
        font=dict(size=11, color="#34495E"),
        align='left',
        bgcolor='rgba(255, 255, 255, 0.9)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1,
        borderpad=6
    )

# Update layout with improved styling
fig.update_layout(
    height=1000,
    width=1000,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=1.0,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="rgba(0,0,0,0.1)",
        borderwidth=1,
        orientation="h",
        itemwidth=70,
        itemsizing="constant"
    ),
    margin=dict(l=60, r=60, t=100, b=60),
    paper_bgcolor='white',
    plot_bgcolor='rgba(240,240,240,0.3)',
    font=dict(family="Arial", size=12)
)

# Update axes with focused ranges and improved styling
for idx, (metric_key, metric) in enumerate(metrics.items(), 1):
    fig.update_xaxes(
        title_text=f"{metric['name']} %",
        title_font=dict(size=13, color="#2C3E50"),
        range=metric['range'],
        gridcolor='rgba(0,0,0,0.1)',
        row=idx, col=1,
        tickformat='.1f'
    )
    fig.update_yaxes(
        title_text="Count",
        title_font=dict(size=13, color="#2C3E50"),
        gridcolor='rgba(0,0,0,0.1)',
        row=idx, col=1
    )

# Show the figure
fig.show() 

# Print summary statistics
print("\nEmail Marketing Metrics Analysis")
print("=" * 40)
print(f"Number of campaigns analyzed: {len(df):,}")

for metric_key, metric in metrics.items():
    print(f"\n{metric['name']} Performance Thresholds:")
    for label, value in metric['percentiles'].items():
        print(f"{label}: {value:.1f}%")
    print(f"\n{metric['name']} Key Statistics:")
    print(f"Mean: {metric['data'].mean():.1f}%")
    print(f"Standard Deviation: {metric['data'].std():.1f}%")
    print(f"Range: {metric['data'].min():.1f}% - {metric['data'].max():.1f}%")


Email Marketing Metrics Analysis
Number of campaigns analyzed: 300

CTR Performance Thresholds:
25th: 12.2%
Median: 13.8%
75th: 14.9%

CTR Key Statistics:
Mean: 13.7%
Standard Deviation: 1.7%
Range: 10.3% - 17.9%

Open Rate Performance Thresholds:
25th: 28.3%
Median: 30.1%
75th: 31.9%

Open Rate Key Statistics:
Mean: 30.0%
Standard Deviation: 2.5%
Range: 21.6% - 34.9%


In [10]:
def load_and_prepare_data(file_path):
    """Load and prepare the campaign performance data."""
    df = pd.read_csv(file_path, parse_dates=['date_sent'])
    
    # Calculate metrics
    df['open_rate'] = df['n_open'] / df['n_sent']
    df['ctr'] = df['n_click'] / df['n_open']
    
    # Add time components
    df['quarter'] = df['date_sent'].dt.to_period('Q')
    df['month'] = df['date_sent'].dt.month
    df['year'] = df['date_sent'].dt.year
    
    return df

def create_volume_engagement_plot(file_path):
    """Create a plot comparing send volume with engagement metrics over time."""
    # Load and prepare data
    df = pd.read_csv(file_path, parse_dates=['date_sent'])
    df['open_rate'] = df['n_open'] / df['n_sent']
    df['ctr'] = df['n_click'] / df['n_open']
    df['quarter'] = df['date_sent'].dt.to_period('Q')
    
    # Create quarterly aggregates
    quarterly = df.groupby('quarter').agg({
        'n_sent': 'sum',
        'open_rate': 'mean',
        'ctr': 'mean'
    }).reset_index()
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add send volume bars
    fig.add_trace(
        go.Bar(
            x=quarterly['quarter'].astype(str),
            y=quarterly['n_sent'],
            name='Total Sent',
            marker_color='rgba(135, 206, 250, 0.8)',  # Light blue
            showlegend=True
        ),
        secondary_y=False
    )
    
    # Add Open Rate line
    fig.add_trace(
        go.Scatter(
            x=quarterly['quarter'].astype(str),
            y=quarterly['open_rate'],
            name='Open Rate',
            line=dict(color='#E74C3C', width=2),
            mode='lines+markers'
        ),
        secondary_y=True
    )
    
    # Add CTR line
    fig.add_trace(
        go.Scatter(
            x=quarterly['quarter'].astype(str),
            y=quarterly['ctr'],
            name='CTR',
            line=dict(color='#2ECC71', width=2),
            mode='lines+markers'
        ),
        secondary_y=True
    )
    
    # Update layout
    fig.update_layout(
        title='Send Volume vs Engagement Metrics by Quarter',
        xaxis_title='Quarter',
        template='plotly_white',
        height=600,
        width=1200,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        margin=dict(l=50, r=50, t=80, b=50),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(128,128,128,0.2)',
            dtick=50000,
            range=[0, 400000],  # Fixed range to match reference
            tickfont=dict(size=10),
            title_font=dict(size=12)
        ),
        yaxis2=dict(
            showgrid=False,
            range=[0.10, 0.35],  # Range from 10% to 35%
            tickformat='.0%',
            tickfont=dict(size=10),
            title_font=dict(size=12)
        ),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # Update axes labels
    fig.update_yaxes(
        title_text="Total Emails Sent", 
        secondary_y=False,
        tickprefix="",  # Remove any prefix
        ticksuffix="k",  # Add k suffix
        tickvals=[0, 50000, 100000, 150000, 200000, 250000, 300000, 350000, 400000],
        ticktext=['0', '50k', '100k', '150k', '200k', '250k', '300k', '350k', '400k']
    )
    fig.update_yaxes(title_text="Engagement Rate", secondary_y=True)
    
    # Update x-axis
    fig.update_xaxes(
        tickangle=0,
        tickfont=dict(size=10),
        title_font=dict(size=12)
    )
    
    return fig

def analyze_volume_impact(df):
    """Analyze the impact of send volume on engagement metrics."""
    # Calculate volume segments
    df['volume_segment'] = pd.qcut(df['n_sent'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
    
    # Calculate metrics by volume segment
    volume_analysis = df.groupby('volume_segment').agg({
        'n_sent': ['mean', 'count'],
        'open_rate': ['mean', 'std'],
        'ctr': ['mean', 'std']
    }).round(4)
    
    # Create visualization
    fig = go.Figure()
    
    segments = volume_analysis.index
    
    # Add bars for open rate
    fig.add_trace(go.Bar(
        name='Open Rate',
        x=segments,
        y=volume_analysis['open_rate']['mean'],
        error_y=dict(
            type='data',
            array=volume_analysis['open_rate']['std'],
            visible=True
        ),
        marker_color='#E74C3C',
        opacity=0.7
    ))
    
    # Add bars for CTR
    fig.add_trace(go.Bar(
        name='CTR',
        x=segments,
        y=volume_analysis['ctr']['mean'],
        error_y=dict(
            type='data',
            array=volume_analysis['ctr']['std'],
            visible=True
        ),
        marker_color='#2ECC71',
        opacity=0.7
    ))
    
    # Update layout
    fig.update_layout(
        title='Engagement Metrics by Send Volume Segment',
        xaxis_title='Send Volume Segment',
        yaxis_title='Rate',
        template='plotly_white',
        height=500,
        width=1000,
        barmode='group',
        yaxis_tickformat='.1%'
    )
    
    return fig, volume_analysis

def create_daily_trend_analysis(df):
    """Create daily trend analysis with smoothed curves."""
    # Calculate daily metrics
    daily_stats = df.groupby('date_sent').agg({
        'n_sent': 'sum',
        'open_rate': 'mean',
        'ctr': 'mean'
    }).reset_index()
    
    # Apply smoothing
    window = min(15, len(daily_stats) - (len(daily_stats) % 2 - 1))
    if window > 2:
        daily_stats['ctr_smooth'] = savgol_filter(daily_stats['ctr'], window, 3)
        daily_stats['open_rate_smooth'] = savgol_filter(daily_stats['open_rate'], window, 3)
    else:
        daily_stats['ctr_smooth'] = daily_stats['ctr'].rolling(window=3, center=True).mean()
        daily_stats['open_rate_smooth'] = daily_stats['open_rate'].rolling(window=3, center=True).mean()
    
    # Create figure
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add send volume bars
    fig.add_trace(
        go.Bar(
            x=daily_stats['date_sent'],
            y=daily_stats['n_sent'],
            name='Daily Sends',
            marker_color='#3498DB',
            opacity=0.3
        ),
        secondary_y=False
    )
    
    # Add engagement metrics
    fig.add_trace(
        go.Scatter(
            x=daily_stats['date_sent'],
            y=daily_stats['open_rate_smooth'],
            name='Open Rate (Smoothed)',
            line=dict(color='#E74C3C', width=2)
        ),
        secondary_y=True
    )
    
    fig.add_trace(
        go.Scatter(
            x=daily_stats['date_sent'],
            y=daily_stats['ctr_smooth'],
            name='CTR (Smoothed)',
            line=dict(color='#2ECC71', width=2)
        ),
        secondary_y=True
    )
    
    # Update layout
    fig.update_layout(
        title='Daily Send Volume and Engagement Trends',
        xaxis_title='Date',
        template='plotly_white',
        height=600,
        width=1000,
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    fig.update_yaxes(title_text="Daily Emails Sent", secondary_y=False)
    fig.update_yaxes(title_text="Engagement Rate", tickformat='.1%', secondary_y=True)
    
    return fig

def main():
    # Create and show the plot
    fig = create_volume_engagement_plot('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv')
    fig.show()

if __name__ == "__main__":
    main() 

In [11]:
def add_rolling_average(df, window=30):
    """
    Adds a rolling average column to the dataframe for key metrics.
    This helps visualize seasonal trends and smoother performance changes.
    """
    df = df.sort_values('date_sent')  # Ensure dates are in order
    df['rolling_open_rate'] = df['open_rate'].rolling(window, min_periods=1).mean()
    df['rolling_ctr'] = df['ctr'].rolling(window, min_periods=1).mean()
    return df

# Apply the function to your data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', parse_dates=['date_sent'])
df['open_rate'] = df['n_open'] / df['n_sent']
df['ctr'] = np.where(df['n_open'] > 0, df['n_click'] / df['n_open'], 0)

df = add_rolling_average(df, window=30)

# Plot the actual vs rolling average to highlight trends
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['date_sent'], y=df['open_rate'],
    mode='markers', name='Actual Open Rate',
    marker=dict(color='rgba(52, 152, 219, 0.7)')
))
fig.add_trace(go.Scatter(
    x=df['date_sent'], y=df['rolling_open_rate'],
    mode='lines', name='30-day Rolling Open Rate',
    line=dict(color='rgb(231, 76, 60)', width=3)
))
fig.update_layout(
    title="Open Rate vs 30-Day Rolling Average",
    xaxis_title="Date",
    yaxis_title="Open Rate",
    template="plotly_white"
)
fig.show()

In [12]:
def plot_volume_vs_performance(df):
    """
    Plot a scatter plot to visualize the correlation between the number
    of emails sent (n_sent) and the open rate. A trendline is added to
    understand the relationship more clearly.
    """
    import plotly.express as px
    # Create scatter plot with an OLS trendline
    fig = px.scatter(df,
                     x='n_sent',
                     y='open_rate',
                     trendline='ols',
                     title='Email Volume vs Open Rate',
                     labels={'n_sent': 'Total Emails Sent', 'open_rate': 'Open Rate'})
    fig.show()

# Example usage:
# Assuming df is already loaded and prepared
plot_volume_vs_performance(df)

In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load and prepare data
df = pd.read_csv('Felipe Chaves_TakeHome Exercise - Campaign_Performance.csv', parse_dates=['date_sent'])
df['open_rate'] = df['n_open'] / df['n_sent']
# Use np.where to avoid divide-by-zero for ctr calculation
df['ctr'] = np.where(df['n_open'] > 0, df['n_click'] / df['n_open'], 0)
df = df.sort_values('date_sent')

# Add rolling averages over 30 days
df['rolling_open_rate'] = df['open_rate'].rolling(window=30, min_periods=1).mean()
df['rolling_ctr'] = df['ctr'].rolling(window=30, min_periods=1).mean()

# Create two sets of traces: one for open_rate and one for ctr analysis

# Traces for Open Rate
trace_open_actual = go.Scatter(
    x=df['date_sent'], y=df['open_rate'],
    mode='markers',
    marker=dict(color='blue', opacity=0.5),
    name='Actual Open Rate'
)
trace_open_rolling = go.Scatter(
    x=df['date_sent'], y=df['rolling_open_rate'],
    mode='lines',
    line=dict(color='red', width=3),
    name='30-day Rolling Open Rate'
)

# Traces for CTR
trace_ctr_actual = go.Scatter(
    x=df['date_sent'], y=df['ctr'],
    mode='markers',
    marker=dict(color='green', opacity=0.5),
    name='Actual CTR'
)
trace_ctr_rolling = go.Scatter(
    x=df['date_sent'], y=df['rolling_ctr'],
    mode='lines',
    line=dict(color='orange', width=3),
    name='30-day Rolling CTR'
)

# Create figure and add all traces (but later use update menus to show/hide appropriately)
fig = go.Figure()

# Initially, show the Open Rate traces
fig.add_trace(trace_open_actual)
fig.add_trace(trace_open_rolling)
fig.add_trace(trace_ctr_actual)
fig.add_trace(trace_ctr_rolling)

# For the update menu, define which traces to show for each selection:
# Order of traces: 0 -> open_actual, 1 -> open_rolling, 2 -> ctr_actual, 3 -> ctr_rolling

buttons = [
    dict(
        label="Open Rate",
        method="update",
        args=[{"visible": [True, True, False, False]},
              {"title": "Email Open Rate vs 30-Day Rolling Average",
               "yaxis": {"title": "Open Rate"}}]
    ),
    dict(
        label="CTR",
        method="update",
        args=[{"visible": [False, False, True, True]},
              {"title": "Email CTR vs 30-Day Rolling Average",
               "yaxis": {"title": "CTR"}}]
    )
]

# Update layout with a date range slider, update menu, and title style
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.02,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )],
    title="Email Open Rate vs 30-Day Rolling Average",
    xaxis=dict(
        title="Date Sent",
        rangeslider=dict(visible=True),
        type="date"
    ),
    yaxis=dict(
        title="Open Rate"
    ),
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()